In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pyexasol

# fold

In [3]:
raw_fold = 'D:/data/processing/urea/raw'
df_fold = 'D:/data/processing/urea/df'
an_fold = 'D:/data/processing/urea/an'
folds = {
    'raw_fold':raw_fold, 
    'df_fold':df_fold, 
    'an_fold':an_fold, 
}
folds

{'raw_fold': 'D:/data/processing/urea/raw',
 'df_fold': 'D:/data/processing/urea/df',
 'an_fold': 'D:/data/processing/urea/an'}

# server

In [24]:
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')

# load

## 등록제원 경유, 경유하이브리드

In [11]:
# 1m 21.8s
name = '경유_경유하이브리드_20231206'
file_name = f'{name}.csv'
ch_col = {
    'VIN':'차대번호', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'YRIDNW':'연식', 
    'VHCTY_CD_x':'차종', 
    'FUEL_CD':'연료',
    'VHCL_ERSR_YN':'차량말소유무',
    'FRST_REG_YMD':'최초등록일자',
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'MNFCTR_NM':'제작사명', 
    'EGIN_TY':'엔진형식', 
    'TOTL_WGHT':'총중량', 
    'VHCNM':'차명', 
    
    # 'BSPL_STDG_CD':'법정동코드', 
    # 'MANP_MNG_NO':'제원관리번호', 
    # 'PURPS_CD2':'용도', 
    # 'VHCL_FBCTN_YMD':'제작일자', 
    # 'VHCL_MNG_NO':'차량관리번호', a
    # 'VHRNO':'자동차등록번호',
    # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
    # 'VHCL_OWNR_CL_CD':'소유자구분',  
}
cs = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, usecols=ch_col.keys())
# cs = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, nrows=10)
csr = cs.rename(columns=ch_col)
csr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11905778 entries, 0 to 11905777
Data columns (total 12 columns):
 #   Column    Dtype  
---  ------    -----  
 0   배출가스인증번호  object 
 1   배출가스등급    object 
 2   최초등록일자    object 
 3   차량말소유무    object 
 4   연식        int64  
 5   차종        object 
 6   차대번호      object 
 7   엔진형식      object 
 8   연료        object 
 9   제작사명      object 
 10  총중량       float64
 11  차명        object 
dtypes: float64(1), int64(1), object(10)
memory usage: 1.1+ GB


In [12]:
csr['차명'].isnull().sum()

0

## 등록정보

In [ ]:
# # 1m 21.8s
# name = 'ods_ceg_car_mid'
# file_name = f'{name}.csv'
# # STD
# ch_col = {
#     'VIN':'차대번호', 
#     'EXHST_GAS_GRD_CD':'배출가스등급', 
#     'EXHST_GAS_CERT_NO':'배출가스인증번호',
#     'VHCL_ERSR_YN':'차량말소유무',
#     'MANP_MNG_NO':'제원관리번호', 
#     'YRIDNW':'연식', 
#     'VHCTY_CD':'차종', 
#     'PURPS_CD2':'용도', 
#     'FRST_REG_YMD':'최초등록일자',

#     # 'BSPL_STDG_CD':'법정동코드', 
#     # 'VHRNO':'자동차등록번호',
#     # 'VHCL_FBCTN_YMD':'제작일자', 
#     # 'VHCL_MNG_NO':'차량관리번호', 
#     # 'EXTGAS_INSP_VLD_YMD':'검사유효일',
#     # 'VHCL_OWNR_CL_CD':'소유자구분', 
# }

# # ODS
# ch_col = {
#     'VIN':'차대번호', 
#     'EMIS_GRD':'배출가스등급', 
#     'EMIS_CRTCNO':'배출가스인증번호',
#     'VHCL_ERSR_YN':'차량말소유무',
#     'SRCMNNO':'제원관리번호', 
#     'VHCL_YRIDNW':'연식', 
#     'VHCTY':'차종', 
#     'PURPS':'용도', 
#     'FST_REG_DE':'최초등록일자',

#     # 'BSPL_LEDO_CD':'법정동코드', 
#     # 'VHRNO':'자동차등록번호',
#     # 'VHCL_MNFCT_DE':'제작일자', 
#     # 'VHMNO':'차량관리번호', 
#     # 'EMIS_INSP_EFCT_DE':'검사유효일',
#     # 'OWNR_SE':'소유자구분', 
# }
# car = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, usecols=ch_col.keys())
# carr = car.rename(columns=ch_col)
# carr.info()

In [149]:
# carr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량'],
      dtype='object')

In [13]:
csr['배출가스등급'].unique()

array(['A0505', 'A0503', 'A0504', nan], dtype=object)

In [14]:
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
csr['배출가스등급'] = csr['배출가스등급'].replace(grd_dict)
csr['배출가스등급'].unique()

array(['5', '3', '4', nan], dtype=object)

In [15]:
# 4.0s
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
csr['차종'] = csr['차종'].replace(cd_dict)
csr['차종'].unique()

array(['화물', '승용', '승합', '특수'], dtype=object)

In [ ]:
# ## 용도 코드 변환
# purps_dict = {
#     'A08P':'개인용', 
#     'A08B':'영업용', 
#     'A08O':'관용',
# }
# csr['용도'] = csr['용도'].replace(purps_dict)
# csr['용도'].unique()

In [17]:
csr.shape

(11905778, 12)

In [16]:
# 차대번호 중복 제거
csr['최초등록일자'] = pd.to_numeric(csr['최초등록일자'], errors='coerce')
csr['최초등록일자'].dtype
# 19.4s
csrm = csr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
csrm.shape

(11902152, 12)

## 제원정보

In [ ]:
# # 1m 21.8s
# name = 'ods_ceg_car_src_mig'
# file_name = f'{name}.csv'
# # STD
# ch_col = {
#     'MANP_MNG_NO':'제원관리번호', 
#     'FUEL_CD':'연료',
#     'VHCTY_TY_CD2':'차종유형', 
#     'MNFCTR_NM':'제작사명', 
#     'VHCNM':'차명', 
#     'VHCL_FRM':'자동차형식', 
#     'EGIN_TY':'엔진형식', 
#     'TOTL_WGHT':'총중량',

#     # 'VHCTY_CL_CD':'차종분류',
#     # 'CRYNG_WGHT':'적재중량',
#     # 'DSPLVL':'배기량', 
#     # 'EGIN_OTPT':'엔진출력', 
# }

# # ODS
# ch_col = {
#     'SRCMNNO':'제원관리번호', 
#     'FUEL':'연료',
#     'VHCTY_TY':'차종유형', 
#     'MKR_NM':'제작사명', 
#     'VHCNM':'차명', 
#     'CAR_FRM':'자동차형식', 
#     'EGINTY':'엔진형식', 
#     'TOT_WGHT':'총중량',

#     # 'VHCTY_CL':'차종분류',
#     # 'CRYNG_WGHT':'적재중량',
#     # 'DSPLVL':'배기량', 
#     # 'EGINPWR':'엔진출력', 
# }
# src = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, usecols=ch_col.keys())
# srcr = src.rename(columns=ch_col)
# srcr.info()

In [18]:
# 7.9s
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
csrm['연료'] = csrm['연료'].replace(fuel_dict)
csrm['연료'].unique()

array(['경유', '경유 하이브리드'], dtype=object)

In [ ]:
# ## 차종유형 코드 변환
# ty_dict = {
#     'A30C':'경형', 
#     'A30L':'대형', 
#     'A30M':'중형', 
#     'A30S':'소형',
# }
# csrm['차종유형'] = csrm['차종유형'].replace(ty_dict)

## 등록&제원 병합

In [ ]:
# srcr.shape, len(srcr['제원관리번호'].unique())

In [ ]:
# carm.shape, len(carm['차대번호'].unique())

In [ ]:
# cs = carm.merge(srcr, on='제원관리번호', how='left')
# cs.shape

## 경유차량, 말소제거, 중복차대 제거

In [20]:
csrm.shape

(11902152, 12)

In [21]:
# 차대번호 중복 제거
csrm['최초등록일자'] = pd.to_numeric(csrm['최초등록일자'], errors='coerce')
csrm['최초등록일자'].dtype
# 19.4s
csrm = csrm.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
csrm.shape

(11902152, 12)

In [22]:
# 5.0s
csm = csrm[(csrm['연료'] == '경유') & (csrm['차량말소유무'] == 'N')].reset_index(drop=True)
csm.shape

(9533735, 12)

## 저감장치 부착이력

In [27]:
# 3.0s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
# STD
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분',
    # 'RDCDVC_KND_CD':'저감장치종류',
}

# # ODS
# att_ch_col = {
#     'VIN':'차대번호', 
#     'RDCDVC_SE':'저감장치구분',
#     # 'RDCDVC_KND':'저감장치종류',
# }
attr = att.rename(columns=att_ch_col)
print(attr.columns)
print('data load : STD_DLM_TB_ERP_ATT_HIS')

Index(['차대번호', '저감장치구분'], dtype='object')
data load : STD_DLM_TB_ERP_ATT_HIS


In [28]:
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '2종', '엔진교체', '삼원촉매', '1종+SCR'], dtype=object)

In [29]:
attr.shape, len(attr['차대번호'].unique())

((1015941, 2), 1014369)

In [30]:
sorted(['1종', '3종', '엔진개조', '2종', '엔진교체', '삼원촉매', '1종+SCR'])

['1종', '1종+SCR', '2종', '3종', '삼원촉매', '엔진개조', '엔진교체']

In [31]:
attr.sort_values('저감장치구분').head()

,차대번호,저감장치구분
0,KL2UR52FDXP016197,1종
719750,KMFWVH7HP6U730861,1종
719751,KMFLA18EP7C019574,1종
719752,KMFLA18EP7C026572,1종
719754,KMFWVH7JP5U647739,1종


In [32]:
# 차대번호 중복 제거
attm = attr.sort_values('저감장치구분').drop_duplicates('차대번호').reset_index(drop=True)
attm.shape

(1014369, 2)

## 인증대장

In [33]:
name = '인증대장(대형)_SCR_적용차종(EURO 4, 5)'
file_name = f'{name}.xlsx'
l45 = pd.read_excel(os.path.join(raw_fold, file_name))
l45.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   제작사         42 non-null     object 
 1   차명          42 non-null     object 
 2   자동차형식       42 non-null     object 
 3   엔진형식        42 non-null     object 
 4   배기량         42 non-null     float64
 5   최대출력        43 non-null     object 
 6   Unnamed: 6  43 non-null     object 
 7   변속기         42 non-null     object 
 8   총중량         42 non-null     object 
 9   공차중량        42 non-null     object 
 10  제작사.1       42 non-null     object 
 11  제작국         42 non-null     object 
 12  인증번호        43 non-null     object 
 13  인증일자        43 non-null     object 
 14  비고          42 non-null     object 
dtypes: float64(1), object(14)
memory usage: 5.2+ KB


In [34]:
l45_col = [
    '제작사1',
    '차명',
    '자동차형식',
    '엔진형식',
    '배기량',
    '최대출력_PS',
    '최대출력_RPM',
    '변속기',
    '총중량',
    '공차중량',
    '제작사2',
    '제작국',
    '인증번호',
    '인증일자',
    '비고',
]
l45.columns = l45_col
l45.columns

Index(['제작사1', '차명', '자동차형식', '엔진형식', '배기량', '최대출력_PS', '최대출력_RPM', '변속기',
       '총중량', '공차중량', '제작사2', '제작국', '인증번호', '인증일자', '비고'],
      dtype='object')

In [35]:
l45.shape

(43, 15)

In [36]:
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,NaN,NaN,NaN,NaN,NaN,PS,RPM,NaN,NaN,NaN,NaN,NaN,배출,배출,NaN
1,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
2,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
4,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4


In [37]:
l45 = l45.iloc[1:].reset_index(drop=True)
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
1,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
2,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4
4,타타대우,대우트랙터,EX3TX2,F3BE3681B,12882.0,560,1900,자동12단,26020,8810,타타대우상용차,한국,8MY-DC-24-15,2008-06-30 00:00:00,유로4


In [38]:
name = '인증대장(대형)_SCR_적용차종(EURO 6)'
file_name = f'{name}.xlsx'
l6 = pd.read_excel(os.path.join(raw_fold, file_name))
l6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   No           21 non-null     int64 
 1   제작사          21 non-null     object
 2   인증번호         21 non-null     object
 3   인증일자         21 non-null     object
 4   차명           21 non-null     object
 5   차종           21 non-null     object
 6   차형식          21 non-null     object
 7   엔진형식         21 non-null     object
 8   배기량          21 non-null     int64 
 9   변속기          21 non-null     object
 10  최대출력         21 non-null     int64 
 11  Unnamed: 11  21 non-null     int64 
 12  총중량          21 non-null     object
 13  공차중량         21 non-null     object
 14  제작국          21 non-null     object
 15  사용연료         21 non-null     object
 16  배출기준         21 non-null     object
 17  유럽기준         21 non-null     object
 18  SCR유무        21 non-null     object
dtypes: int64(4), object(15)
memory 

In [39]:
name = '인증대장(소형)_SCR_적용차종(EURO 6)'
file_name = f'{name}.xlsx'
s6 = pd.read_excel(os.path.join(raw_fold, file_name))
s6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   일련번호          47 non-null     int64 
 1   제작사           47 non-null     object
 2   배출가스차종        47 non-null     object
 3   배출가스인증번호      47 non-null     object
 4   배출가스인증일       47 non-null     object
 5   차명            47 non-null     object
 6   자동차형식         47 non-null     object
 7   엔진형식          47 non-null     object
 8   배기량           47 non-null     int64 
 9   변속기           47 non-null     object
 10  최대출력PS        47 non-null     int64 
 11  최대출력RPM       47 non-null     int64 
 12  총중량           47 non-null     int64 
 13  공차중량          47 non-null     int64 
 14  제작국가          47 non-null     object
 15  사용연료          47 non-null     object
 16  배출가스기준        47 non-null     object
 17  유럽 기준         47 non-null     object
 18  SCR 유(Y)무(N)  47 non-null     object
dtypes: int64(6

In [40]:
# 확인된 사항
# l45, l6, s6 배출가스인증번호(인증번호) 중복 없음

In [41]:
len(set(l45['인증번호'].to_list()))

42

In [42]:
len(set(l45['인증번호'].to_list()) - set(l6['인증번호'].to_list()))

42

In [43]:
len(set(l6['인증번호'].to_list()))

21

In [44]:
len(set(l6['인증번호'].to_list()) - set(s6['배출가스인증번호'].to_list()))

21

In [45]:
len(set(l45['인증번호'].to_list()) - set(s6['배출가스인증번호'].to_list()))

42

## 제작사 확인

In [46]:
name = '엔진형식_제작사명_리스트(수정)'
file_name = f'{name}.xlsx'
ck = pd.read_excel(os.path.join(raw_fold, file_name))
ck.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679 entries, 0 to 8678
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   엔진형식                 8679 non-null   object
 1   제작사명                 8679 non-null   object
 2   차대번호                 8679 non-null   int64 
 3   국내/외 구분(1:국내, 2:국외)  8679 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 271.3+ KB


In [47]:
ck.head()

,엔진형식,제작사명,차대번호,"국내/외 구분(1:국내, 2:국외)"
0,-,(주) 두성특장차,1,1
1,-,한국특장차(주),1,1
2,.D4GB,(주) 골드밴,1,1
3,0,SOFA차량,4,1
4,0,남도수입,1,2


In [48]:
ck = ck.rename(columns={'국내/외 구분(1:국내, 2:국외)':'구분'})
ck['구분'] = ck['구분'].replace({1:'국내', 2:'국외'})
ck.head()

,엔진형식,제작사명,차대번호,구분
0,-,(주) 두성특장차,1,국내
1,-,한국특장차(주),1,국내
2,.D4GB,(주) 골드밴,1,국내
3,0,SOFA차량,4,국내
4,0,남도수입,1,국외


In [49]:
ck = ck.drop(['차대번호'], axis=1)
ck.head()

,엔진형식,제작사명,구분
0,-,(주) 두성특장차,국내
1,-,한국특장차(주),국내
2,.D4GB,(주) 골드밴,국내
3,0,SOFA차량,국내
4,0,남도수입,국외


In [50]:
ck.shape, ck.drop_duplicates(['엔진형식', '제작사명']).shape

((8679, 3), (8679, 3))

## 220630_기준_SCR_부착_차량_통계자료_수정

In [51]:
scr = pd.read_excel(os.path.join(raw_fold, '220630_기준_SCR_부착_차량_통계자료_수정.xlsx'))
scr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   인증번호      501 non-null    object
 1   차종        500 non-null    object
 2   원본차종      500 non-null    object
 3   제조사       500 non-null    object
 4   차명        351 non-null    object
 5   통계자료존재유무  501 non-null    object
dtypes: object(6)
memory usage: 23.6+ KB


In [52]:
scr = scr[['인증번호', '차종', '제조사', '차명', '통계자료존재유무']]
scr.shape

(501, 5)

In [53]:
scr['인증번호'].isnull().sum()

0

In [54]:
scr.shape, len(scr['인증번호'].unique())

((501, 5), 501)

# 전처리
- 컬럼명 수정
- 필수 컬럼만 추출
- 등록 + 인증대장 + 저감
- 등록 + 인증대장 + 저감 + SCR 통계자료(차종, 제조사, 차명 우선)

## 인증대장 병합

In [55]:
l45.columns.to_list()

['제작사1',
 '차명',
 '자동차형식',
 '엔진형식',
 '배기량',
 '최대출력_PS',
 '최대출력_RPM',
 '변속기',
 '총중량',
 '공차중량',
 '제작사2',
 '제작국',
 '인증번호',
 '인증일자',
 '비고']

In [56]:
l45.head()

,제작사1,차명,자동차형식,엔진형식,배기량,최대출력_PS,최대출력_RPM,변속기,총중량,공차중량,제작사2,제작국,인증번호,인증일자,비고
0,현대자동차,현대25.5톤덤프트럭,HD255-YDU0-DHECA,D6CE,12742.0,520,1700,자동12단,38730,13230,현대자동차,한국,AMY-HD-14-35,2010-07-19 00:00:00,유로5
1,타타대우,대우25톤장축카고트럭,BT9CC1,DV11,10964.0,440,1900,수동16단,38800,13670,타타대우상용차,한국,7MY-DC-23-70,2007-12-18 00:00:00,유로4
2,타타대우,대우5톤초장축카고트럭,BC4AH1,F4AE3681D,5880.0,251,2700,수동6단,11415,6285,타타대우상용차,한국,7MY-DC-23-72,2007-12-18 00:00:00,유로4
3,타타대우,대우5톤초장축카고트럭,BC4AT1,DL06,5890.0,270,2500,수동6단,11500,6370,타타대우상용차,한국,8MY-DC-24-12,2008-05-21 00:00:00,유로4
4,타타대우,대우트랙터,EX3TX2,F3BE3681B,12882.0,560,1900,자동12단,26020,8810,타타대우상용차,한국,8MY-DC-24-15,2008-06-30 00:00:00,유로4


In [57]:
l45m = l45[[
    '인증번호',
    '제작사2',
    '차명',
    '제작국',
    '비고'

    # '제작사1',
    # '자동차형식',
    # '엔진형식',
    # '배기량',
    # '최대출력_PS',
    # '최대출력_RPM',
    # '변속기',
    # '총중량',
    # '공차중량',
    # '인증일자',
]]
l45m.head()

,인증번호,제작사2,차명,제작국,비고
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4


In [58]:
l45m = l45m.rename(columns={'인증번호':'배출가스인증번호', '제작사2':'제작사', '제작국가':'제작사', '비고':'유럽기준'})
l45m['SCR유무'] = 'Y'
l45m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5,Y
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4,Y
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4,Y


In [59]:
l6.head()

,No,제작사,인증번호,인증일자,차명,차종,차형식,엔진형식,배기량,변속기,최대출력,Unnamed: 11,총중량,공차중량,제작국,사용연료,배출기준,유럽기준,SCR유무
0,6,자일대우버스(주),LMY-DB-14-105,2020-12-31,BX212,승용(초대형),BX212M-GPLBZC441L,OM470LA.6-6,10677,자동화수동변속기,456,1600,16035,13110,한국,경유(Diesel),2017년 10월 기준,유로6,Y
1,8,타타대우상용차(주),LMY-DC-14-87,2020-11-03,LD트럭,화물(대형),FE5BC1,F4AFE411C,4485,수동6단,206,2500,9910,4715,한국,경유(Diesel),2017년 10월 기준,유로6,Y
2,12,타타대우상용차(주),KMY-DC-14-82,2019-11-26,프리마14톤장축카고트럭,화물(초대형),JL8A82,DX12P,11051,자동12단,460,1900,25800(25260),11670(11130),한국,경유(Diesel),2017년 10월 기준,유로6,Y
3,17,(주)우진산전,GMY-WJ-14-02,2016-01-06,우진산전 굴절버스,승용(대형),KM9ZGCZE,ISB6.7E6,6690,없음,280,2100,25995,19495,한국,경유-하이브리드(D-Hybrid),2014년 1월 기준,유로6,Y
4,19,타타대우상용차(주),GMY-DC-14-52,2016-11-07,노부스5톤극초장축플러스8.6카고트럭,화물(대형),DC8CT7,DL06P,5890,수동 9단,280,2500,12500(12710),7370(7580),한국,경유(Diesel),2014년 1월 기준,유로6,Y


In [60]:
l6.columns.to_list()

['No',
 '제작사',
 '인증번호',
 '인증일자',
 '차명',
 '차종',
 '차형식',
 '엔진형식',
 '배기량',
 '변속기',
 '최대출력',
 'Unnamed: 11',
 '총중량',
 '공차중량',
 '제작국',
 '사용연료',
 '배출기준',
 '유럽기준',
 'SCR유무']

In [61]:
l6m = l6[[
    '인증번호',
    '제작사',
    '차명',
    '제작국',
    '유럽기준',
    'SCR유무',
    
    # 'No',
    # '인증일자',
    # '차종',
    # '차형식',
    # '엔진형식',
    # '배기량',
    # '변속기',
    # '최대출력',
    # 'Unnamed: 11',
    # '총중량',
    # '공차중량',
    # '사용연료',
    # '배출기준',
]]
l6m.head()

,인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,LMY-DB-14-105,자일대우버스(주),BX212,한국,유로6,Y
1,LMY-DC-14-87,타타대우상용차(주),LD트럭,한국,유로6,Y
2,KMY-DC-14-82,타타대우상용차(주),프리마14톤장축카고트럭,한국,유로6,Y
3,GMY-WJ-14-02,(주)우진산전,우진산전 굴절버스,한국,유로6,Y
4,GMY-DC-14-52,타타대우상용차(주),노부스5톤극초장축플러스8.6카고트럭,한국,유로6,Y


In [62]:
l6m = l6m.rename(columns={'인증번호':'배출가스인증번호'})
l6m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,LMY-DB-14-105,자일대우버스(주),BX212,한국,유로6,Y
1,LMY-DC-14-87,타타대우상용차(주),LD트럭,한국,유로6,Y
2,KMY-DC-14-82,타타대우상용차(주),프리마14톤장축카고트럭,한국,유로6,Y
3,GMY-WJ-14-02,(주)우진산전,우진산전 굴절버스,한국,유로6,Y
4,GMY-DC-14-52,타타대우상용차(주),노부스5톤극초장축플러스8.6카고트럭,한국,유로6,Y


In [63]:
s6.head()

,일련번호,제작사,배출가스차종,배출가스인증번호,배출가스인증일,차명,자동차형식,엔진형식,배기량,변속기,최대출력PS,최대출력RPM,총중량,공차중량,제작국가,사용연료,배출가스기준,유럽 기준,SCR 유(Y)무(N)
0,27,현대자동차(주),승용(중형),FMY-HD-14-38,2015-07-29,그랜드스타렉스 2.5디젤 12인(4WD),TQ-25D12-E6-4W5A,D4CB,2497,자동5단,175,3600,3395,2505,한국,경유(Diesel),2014년 1월 기준,유로6,Y
1,29,현대자동차(주),승용(소형),HMY-HD-14-61,2017-12-15,G80 2.2디젤 2WD,DH-22D-H6-2W8A,D4HC,2199,자동8단,202,3800,2435,2025,한국,경유(Diesel),2017년 10월 기준,유로6,Y
2,31,현대자동차(주),승용(소형),HMY-HD-14-63,2017-12-15,G70 2.2 디젤 2WD,IK-22D-E6C-2W8A,D4HC,2199,자동8단,202,3800,2140,1765,한국,경유(Diesel),2017년 10월 기준,유로6,Y
3,32,현대자동차(주),승용(소형),JMY-HD-14-15,2018-02-02,싼타페 2.0디젤 2WD,TM-20D7-H6-2W8A,D4HA,1995,자동8단,186,4000,2485,1910,한국,경유(Diesel),2017년 10월 기준,유로6,Y
4,33,현대자동차(주),승용(소형),HMY-HD-14-62,2017-12-15,G70 2.2 디젤 AWD,IK-22D-E6C-AW8A,D4HC,2199,자동8단,202,3800,2215,1840,한국,경유(Diesel),2017년 10월 기준,유로6,Y


In [64]:
s6.columns.to_list()

['일련번호',
 '제작사',
 '배출가스차종',
 '배출가스인증번호',
 '배출가스인증일',
 '차명',
 '자동차형식',
 '엔진형식',
 '배기량',
 '변속기',
 '최대출력PS',
 '최대출력RPM',
 '총중량',
 '공차중량',
 '제작국가',
 '사용연료',
 '배출가스기준',
 '유럽 기준',
 'SCR 유(Y)무(N)']

In [65]:
s6m = s6[[
    '배출가스인증번호',
    '제작사',
    '차명',
    '제작국가',
    '유럽 기준',
    'SCR 유(Y)무(N)'

    # '일련번호',
    # '배출가스차종',
    # '배출가스인증일',
    # '자동차형식',
    # '엔진형식',
    # '배기량',
    # '변속기',
    # '최대출력PS',
    # '최대출력RPM',
    # '총중량',
    # '공차중량',
    # '사용연료',
    # '배출가스기준',
]]
s6m.head()

,배출가스인증번호,제작사,차명,제작국가,유럽 기준,SCR 유(Y)무(N)
0,FMY-HD-14-38,현대자동차(주),그랜드스타렉스 2.5디젤 12인(4WD),한국,유로6,Y
1,HMY-HD-14-61,현대자동차(주),G80 2.2디젤 2WD,한국,유로6,Y
2,HMY-HD-14-63,현대자동차(주),G70 2.2 디젤 2WD,한국,유로6,Y
3,JMY-HD-14-15,현대자동차(주),싼타페 2.0디젤 2WD,한국,유로6,Y
4,HMY-HD-14-62,현대자동차(주),G70 2.2 디젤 AWD,한국,유로6,Y


In [66]:
s6m = s6m.rename(columns={'SCR 유(Y)무(N)':'SCR유무', '제작국가':'제작국', '유럽 기준':'유럽기준'})
s6m.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,FMY-HD-14-38,현대자동차(주),그랜드스타렉스 2.5디젤 12인(4WD),한국,유로6,Y
1,HMY-HD-14-61,현대자동차(주),G80 2.2디젤 2WD,한국,유로6,Y
2,HMY-HD-14-63,현대자동차(주),G70 2.2 디젤 2WD,한국,유로6,Y
3,JMY-HD-14-15,현대자동차(주),싼타페 2.0디젤 2WD,한국,유로6,Y
4,HMY-HD-14-62,현대자동차(주),G70 2.2 디젤 AWD,한국,유로6,Y


In [67]:
l45m.shape[0] + l6m.shape[0] + s6m.shape[0]

110

In [68]:
cert = pd.concat([l45m, l6m, s6m], ignore_index=True)
cert.shape

(110, 6)

In [69]:
cert[cert['차명'].str.contains(r'\n')]

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
22,AMY-SK-14-02,스카니아,스카니아트랙터\n(견인형),스웨덴,유로5,Y
23,AMY-SK-14-03,스카니아,스카니아카고트럭\n(견인형),스웨덴,유로5,Y
24,CMY-SK-14-01,스카니아,스카니아카고트럭\n(견인형),스웨덴,유로5,Y


In [70]:
cert['차명'] = cert['차명'].str.replace('\n', '')
cert.loc[22:24, :]

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
22,AMY-SK-14-02,스카니아,스카니아트랙터(견인형),스웨덴,유로5,Y
23,AMY-SK-14-03,스카니아,스카니아카고트럭(견인형),스웨덴,유로5,Y
24,CMY-SK-14-01,스카니아,스카니아카고트럭(견인형),스웨덴,유로5,Y


In [71]:
cert['제작사'].value_counts(dropna=False)

현대자동차(주)      40
타타대우상용차       11
대우버스          10
타타대우상용차(주)     8
현대자동차㈜         7
현대자동차          6
볼보트럭코퍼레이션      5
Daimler AG     5
자일대우버스(주)      5
스카니아           3
만              3
이베코            3
기아(주)          2
진화영맨           1
(주)우진산전        1
Name: 제작사, dtype: int64

In [72]:
cert['제작국'].value_counts(dropna=False)

한국      90
스웨덴      8
독일       8
이탈리아     3
중국       1
Name: 제작국, dtype: int64

In [73]:
cert['유럽기준'].value_counts(dropna=False)

유로6    68
유로5    33
유로4     9
Name: 유럽기준, dtype: int64

In [74]:
cert['SCR유무'].value_counts(dropna=False)

Y    110
Name: SCR유무, dtype: int64

In [75]:
cert.head()

,배출가스인증번호,제작사,차명,제작국,유럽기준,SCR유무
0,AMY-HD-14-35,현대자동차,현대25.5톤덤프트럭,한국,유로5,Y
1,7MY-DC-23-70,타타대우상용차,대우25톤장축카고트럭,한국,유로4,Y
2,7MY-DC-23-72,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
3,8MY-DC-24-12,타타대우상용차,대우5톤초장축카고트럭,한국,유로4,Y
4,8MY-DC-24-15,타타대우상용차,대우트랙터,한국,유로4,Y


In [76]:
cert.isnull().sum()

배출가스인증번호    0
제작사         0
차명          0
제작국         0
유럽기준        0
SCR유무       0
dtype: int64

### [출력] 인증대장

In [77]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231214'

In [63]:
# # 0.0s
# cert.to_csv(os.path.join(raw_fold, f'인증대장_{today_date}.csv'), index=False)
# print(os.path.join(raw_fold, f'인증대장_{today_date}.csv'))

D:/data/processing/urea/raw\인증대장_20231214.csv


### [로드] 인증대장 종합

In [6]:
cert = pd.read_csv(os.path.join(raw_fold, '인증대장_20231214.csv'))
cert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   배출가스인증번호  110 non-null    object
 1   제작사       110 non-null    object
 2   차명        110 non-null    object
 3   제작국       110 non-null    object
 4   유럽기준      110 non-null    object
 5   SCR유무     110 non-null    object
dtypes: object(6)
memory usage: 5.3+ KB


## 등록 + 인증
- 기준 : 배출가스인증번호

In [78]:
csm.shape

(9533735, 12)

In [79]:
cc = csm.copy()

In [80]:
cert.columns

Index(['배출가스인증번호', '제작사', '차명', '제작국', '유럽기준', 'SCR유무'], dtype='object')

In [81]:
# 1m 6.8s
for one, two, three, four, five, six in cert.values:
    cc.loc[cc['배출가스인증번호'] == one, ['제작사명', '차명', '제작국', '유럽기준', 'SCR유무']] = [two, three, four, five, six]    

In [82]:
cc.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준', 'SCR유무'],
      dtype='object')

In [83]:
cc[['제작사명', '차명', '제작국', '유럽기준', 'SCR유무']].isnull().sum()

제작사명         592
차명             0
제작국      8473081
유럽기준     8473081
SCR유무    8473081
dtype: int64

## 등록 + 인증 + 저감

In [84]:
# 13.8s
cca = cc.merge(attm, on='차대번호', how='left')
cca.shape

(9533735, 16)

In [85]:
cca.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분'],
      dtype='object')

### [출력] 제작사명, 엔진형식
- 국내외 구분 위해

In [164]:
ex = cca.groupby(['엔진형식', '제작사명'])['차대번호'].count().reset_index()
ex.shape

(8679, 3)

In [165]:
ex.head()

,엔진형식,제작사명,차대번호
0,-,(주) 두성특장차,1
1,-,한국특장차(주),1
2,.D4GB,(주) 골드밴,1
3,0,SOFA차량,4
4,0,남도수입,1


In [ ]:
exc = ex[['엔진형식', '제작사명']].merge(ck, on=['엔진형식', '제작사명'], how='left')
exc1 = exc[exc['구분'].isnull()].reset_index(drop=True)
exc1.shape

In [ ]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

In [166]:
# ex.to_excel(os.path.join(an_fold, f'엔진형식_제작사명_리스트_{today_date}.xlsx'), index=False)

In [ ]:
exc1.to_excel(os.path.join(an_fold, f'추가_엔진형식_제작사명_리스트_{today_date}.xlsx'), index=False)

## 제작사 국내외 구분 설정

In [86]:
cca.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분'],
      dtype='object')

In [87]:
ck.columns

Index(['엔진형식', '제작사명', '구분'], dtype='object')

In [88]:
cca.shape

(9533735, 16)

In [89]:
ccac = cca.merge(ck, on=['제작사명', '엔진형식'], how='left')
ccac.shape

(9533735, 17)

In [90]:
ccac['구분'].isnull().sum()

429555

In [91]:
ccac.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분', '구분'],
      dtype='object')

## 등록 + 인증 + 저감 + 제작국 + SCR 통계

In [92]:
scr.head()

,인증번호,차종,제조사,차명,통계자료존재유무
0,AMY-HD-14-35,화물,현대,"트라고/엑시언트 (L엔진, D6CE)",Y
1,EMY-HD-14-45,승합,현대,"유니버스 / 엑시언트 (L엔진, D6CG)",Y
2,EMY-HD-14-48,승합,현대,"유니버스,뉴슈퍼에어로시티 / 엑시언트 (H엔진, D6HC)",Y
3,EMY-HD-14-51,승합,현대,"현대그린시티, 이-에어로타운 / 메가트럭, 메가트럭(와이드캡), 파비스 (G엔진, ...",Y
4,EMY-HD-14-53,승합,현대,"뉴카운티 / 마이티 (F엔진, D4GB)",Y


In [93]:
scr.shape

(501, 5)

In [94]:
scr['차명'].isnull().sum()

150

In [95]:
ccac[ccac['배출가스인증번호'].str.contains(' ', regex=False, na=False)]

,배출가스인증번호,배출가스등급,최초등록일자,차량말소유무,연식,차종,차대번호,엔진형식,연료,제작사명,총중량,차명,제작국,유럽기준,SCR유무,저감장치구분,구분


In [96]:
scr[scr['인증번호'].str.contains(' ', regex=False, na=False)]

,인증번호,차종,제조사,차명,통계자료존재유무


In [97]:
ccac['배출가스인증번호'].dtype

dtype('O')

In [98]:
scr['인증번호'].dtype

dtype('O')

In [99]:
ccac.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분', '구분'],
      dtype='object')

In [100]:
ccac[['차종', '제작사명', '차명']].head()

,차종,제작사명,차명
0,승합,현대자동차(주),스타리아 (STARIA)
1,승용,현대자동차(주),투싼(TUCSON)
2,화물,현대자동차㈜,포터 2.5디젤 초장축 더블캡 AT
3,화물,현대자동차㈜,포터 2.5디젤 초장축 더블캡 AT
4,화물,현대자동차㈜,포터 2.5디젤 초장축 더블캡 AT


In [101]:
scr.head()

,인증번호,차종,제조사,차명,통계자료존재유무
0,AMY-HD-14-35,화물,현대,"트라고/엑시언트 (L엔진, D6CE)",Y
1,EMY-HD-14-45,승합,현대,"유니버스 / 엑시언트 (L엔진, D6CG)",Y
2,EMY-HD-14-48,승합,현대,"유니버스,뉴슈퍼에어로시티 / 엑시언트 (H엔진, D6HC)",Y
3,EMY-HD-14-51,승합,현대,"현대그린시티, 이-에어로타운 / 메가트럭, 메가트럭(와이드캡), 파비스 (G엔진, ...",Y
4,EMY-HD-14-53,승합,현대,"뉴카운티 / 마이티 (F엔진, D4GB)",Y


In [102]:
pr = ccac.copy()

In [108]:
scr['차명'].isnull().sum()

150

In [112]:
np.isnan(scr.loc[scr['차명'].isnull(), '차명'].values[0])

True

In [114]:
# 5m 17.2s
for one, two, three, four, five in scr.values: # 배출가스인증번호, 차종, 제조사, 차명, 통계자료존재유무
    if one == 'FMY-DW-14-34': # FMY-DW-14-34의 값(차종, 제조사, 차명)이 비어있음
        pr.loc[pr['배출가스인증번호'] == one, '통계자료존재유무'] = five
    elif not pd.isna(four): # SCR통계에 차명이 존재하는 경우
        pr.loc[pr['배출가스인증번호'] == one, ['차종', '제작사명', '차명', '통계자료존재유무']] = [two, three, four, five]
    else: # SCR통계에 차명이 존재하지 않는 경우
        pr.loc[pr['배출가스인증번호'] == one, ['차종', '제작사명', '통계자료존재유무']] = [two, three, five]

In [115]:
pr['차명'].isnull().sum()

0

In [116]:
pr.shape

(9533735, 18)

In [117]:
pr['통계자료존재유무'].isnull().sum()

7379999

In [118]:
pr[['차종', '제작사명', '차명', '구분']].isnull().sum()

차종           0
제작사명       592
차명           0
구분      429555
dtype: int64

In [119]:
pr.head()

,배출가스인증번호,배출가스등급,최초등록일자,차량말소유무,연식,차종,차대번호,엔진형식,연료,제작사명,총중량,차명,제작국,유럽기준,SCR유무,저감장치구분,구분,통계자료존재유무
0,NaN,NaN,20231129.0,N,2024,승합,KMJYB371BRU160978,D4HB,경유,현대자동차(주),3035.0,스타리아 (STARIA),NaN,NaN,NaN,NaN,국내,NaN
1,NaN,NaN,20231129.0,N,2023,승용,KMHJC81CBPU316203,D4HD,경유,현대자동차(주),1955.0,투싼(TUCSON),NaN,NaN,NaN,NaN,국내,NaN
2,KMY-HD-14-59,3,20231129.0,N,2023,승용,KMFZCZ7KBPU098949,D4CB,경유,현대기아,2965.0,포터 2.5디젤 초장축 더블캡 AT,한국,유로6,Y,NaN,NaN,Y
3,KMY-HD-14-59,3,20231129.0,N,2023,승용,KMFZCZ7KBPU098960,D4CB,경유,현대기아,2965.0,포터 2.5디젤 초장축 더블캡 AT,한국,유로6,Y,NaN,NaN,Y
4,KMY-HD-14-59,3,20231129.0,N,2023,승용,KMFZCZ7KBPU098985,D4CB,경유,현대기아,2965.0,포터 2.5디젤 초장축 더블캡 AT,한국,유로6,Y,NaN,NaN,Y


In [120]:
pr.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분', '구분',
       '통계자료존재유무'],
      dtype='object')

In [121]:
pr.shape

(9533735, 18)

### [출력] 전처리 완료 파일

In [122]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231214'

In [123]:
# 50.2s
pr.to_csv(os.path.join(raw_fold, f'등록_인증_저감_국내외구분_SCR통계참고_{today_date}.csv'), index=False)

### [로드] 전처리 완료 파일

In [65]:
pr = pd.read_csv(os.path.join(raw_fold, '등록_인증_저감_국내외구분_SCR통계참고_20231213.csv'))
pr.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_16144\2004874364.py:1: DtypeWarning: Columns (11,12,13,14,15,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  pr = pd.read_csv(os.path.join(raw_fold, '등록_인증_저감_국내외구분_SCR통계참고_20231213.csv'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9533740 entries, 0 to 9533739
Data columns (total 19 columns):
 #   Column    Dtype  
---  ------    -----  
 0   배출가스인증번호  object 
 1   배출가스등급    object 
 2   최초등록일자    float64
 3   차량말소유무    object 
 4   연식        int64  
 5   차종        object 
 6   차대번호      object 
 7   엔진형식      object 
 8   연료        object 
 9   제작사명      object 
 10  총중량       float64
 11  제작사       object 
 12  차명        object 
 13  제작국       object 
 14  유럽기준      object 
 15  SCR유무     object 
 16  저감장치구분    object 
 17  구분        object 
 18  통계자료존재유무  object 
dtypes: float64(2), int64(1), object(16)
memory usage: 1.3+ GB


In [124]:
pr['차명'].isnull().sum()

0

## 저공해조치 차량과 SCR차량 구분

In [190]:
# 저공해조치차량
    # 저감장치구분 : 1종+SCR인 차량
# SCR차량(EURO6)
    # SCR유무 : Y
    # 국내 : 2017년 이상
    # 국외 : 2015년 이상

In [125]:
mnt = pr[pr['저감장치구분'] == '1종+SCR'].reset_index(drop=True)
mnt.shape

(2372, 18)

### [출력] 저공해조치 차량

In [126]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20231214'

In [127]:
# # 1.7s
# mnt.to_excel(os.path.join(df_fold, f'경유_저공해조치차량_{today_date}.xlsx'), index=False)

In [128]:
pr['구분'].isnull().sum()

429555

In [129]:
euro6 = pr[(pr['저감장치구분'] != '1종+SCR') & ( ((pr['구분'] == '국내') & (pr['연식'] >= 2017)) | ((pr['구분'] == '국외') & (pr['연식'] >= 2015)) | (pr['SCR유무'] == 'Y') )].reset_index(drop=True)
euro6.shape

(4433293, 18)

In [130]:
div = 50 * 10**4
for n in range(euro6.shape[0] // div + 1):
    print(n+1, n*div, (n+1) *div)

1 0 500000
2 500000 1000000
3 1000000 1500000
4 1500000 2000000
5 2000000 2500000
6 2500000 3000000
7 3000000 3500000
8 3500000 4000000
9 4000000 4500000


In [238]:
# # 
# div = 50 * 10**4
# for n in range(euro6.shape[0] // div + 1):
#     # print(n*div, (n+1) *div)
#     euro6.iloc[n*div:(n+1)*div].to_excel(os.path.join(df_fold, f'EURO6차량현황{n+1}_{today_date}.xlsx'), index=False)

# 분석

## SCR 장착 차량 현황

In [131]:
mnt.shape[0] + euro6.shape[0]

4435665

In [132]:
me = pd.concat([mnt, euro6], ignore_index=False)
me.shape

(4435665, 18)

In [133]:
me.columns

Index(['배출가스인증번호', '배출가스등급', '최초등록일자', '차량말소유무', '연식', '차종', '차대번호', '엔진형식',
       '연료', '제작사명', '총중량', '차명', '제작국', '유럽기준', 'SCR유무', '저감장치구분', '구분',
       '통계자료존재유무'],
      dtype='object')

In [134]:
# megrp1 = me.groupby(['배출가스인증번호', '제작사명', '연식', '차종', '차명', '통계자료존재유무'], dropna=False)['차대번호'].count().reset_index()
megrp1 = me.groupby(['배출가스인증번호', '제작사명', '연식', '차종', '차명', '유럽기준', '저감장치구분', '통계자료존재유무'], dropna=False)['차대번호'].count().reset_index()
megrp1 = megrp1.rename(columns={'차대번호':'차량대수'})
megrp1

,배출가스인증번호,제작사명,연식,차종,차명,유럽기준,저감장치구분,통계자료존재유무,차량대수
0,1MY-HD-52,현대자동차(주),2004,화물,현대슈퍼트럭,NaN,1종+SCR,NaN,3
1,2MY-DW-04,타타대우상용차(주),2003,화물,대우25톤카고트럭,NaN,1종+SCR,NaN,1
2,2MY-DW-04,타타대우상용차(주),2004,화물,대우25톤장축카고트럭,NaN,1종+SCR,NaN,1
3,2MY-DW-04,타타대우상용차(주),2004,화물,대우25톤카고트럭,NaN,1종+SCR,NaN,29
4,2MY-DW-04,타타대우상용차(주),2005,화물,대우22.5톤장축카고트럭,NaN,1종+SCR,NaN,1
...,...,...,...,...,...,...,...,...,...
22623,NaN,히아브 만평특장,2022,화물,만평1톤경장비운송차,NaN,NaN,NaN,1
22624,NaN,히아브-테크,2023,화물,히아브테크0.7톤크레인집게차,NaN,NaN,NaN,1
22625,NaN,히아브대종특장(주),2020,화물,히아브대종2.2톤크레인집게차,NaN,NaN,NaN,1
22626,NaN,히아브특장,2023,화물,히아브1.1톤크레인집게차,NaN,NaN,NaN,1


In [135]:
megrp1['차량대수'].sum()

4435665

In [139]:
megrp1.isnull().sum()

배출가스인증번호    19466
제작사명            0
연식              0
차종              0
차명              0
유럽기준        22210
저감장치구분      22326
통계자료존재유무    21435
차량대수            0
dtype: int64

In [142]:
megrp1.loc[megrp1['저감장치구분'].notnull(), '차량대수'].sum()

2372

In [144]:
megrp1.loc[(megrp1['유럽기준'].notnull()) & (megrp1['통계자료존재유무'].isnull()), '차량대수'].sum()

66790

In [145]:
megrp1.loc[(megrp1['유럽기준'].isnull()) & (megrp1['통계자료존재유무'].notnull()), '차량대수'].sum()

1114766

In [146]:
megrp1.loc[(megrp1['유럽기준'].notnull()) | (megrp1['통계자료존재유무'].notnull()), '차량대수'].sum()

2175420

In [136]:
today_date_dtl = today_date = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

'20231214_PM_060101'

### [출력] 경유차량 통계

In [137]:
# 1.9s
megrp1.to_excel(os.path.join(an_fold, f'경유차량_통계_{today_date_dtl}.xlsx'), index=False)

## 저공해조치 차량 현황

In [371]:
mgrp1 = mnt.groupby(['구분'], dropna=False)['차대번호'].count().reset_index()
mgrp1 = mgrp1.rename(columns={'차대번호':'차량대수'})
mgrp1.loc[mgrp1.shape[0], ['구분', '차량대수']] = ['합계', mgrp1['차량대수'].sum()]
mgrp1

,구분,차량대수
0,국내,2102.0
1,국외,270.0
2,합계,2372.0


In [372]:
mgrp2 = mnt.groupby(['구분', '차종'], dropna=False)['차대번호'].count().reset_index()
mgrp2 = mgrp2.rename(columns={'차대번호':'차량대수'})
mgrp2.loc[mgrp2.shape[0], ['구분', '차량대수']] = ['합계', mgrp2['차량대수'].sum()]
mgrp2

,구분,차종,차량대수
0,국내,승합,38.0
1,국내,화물,2064.0
2,국외,화물,270.0
3,합계,NaN,2372.0


In [373]:
mgrp3 = mnt.groupby(['구분', '차종유형'], dropna=False)['차대번호'].count().reset_index()
mgrp3 = mgrp3.rename(columns={'차대번호':'차량대수'})
mgrp3.loc[mgrp3.shape[0], ['구분', '차량대수']] = ['합계', mgrp3['차량대수'].sum()]
mgrp3

,구분,차종유형,차량대수
0,국내,대형,2102.0
1,국외,대형,270.0
2,합계,NaN,2372.0


In [ ]:
mgrp4 = mnt.groupby(['구분', '용도'], dropna=False)['차대번호'].count().reset_index()
mgrp4 = mgrp4.rename(columns={'차대번호':'차량대수'})
mgrp4.loc[mgrp4.shape[0], ['구분', '차량대수']] = ['합계', mgrp4['차량대수'].sum()]
mgrp4

In [374]:
mgrp5 = mnt.groupby(['구분', '차종', '차종유형', '용도'], dropna=False)['차대번호'].count().reset_index()
mgrp5 = mgrp5.rename(columns={'차대번호':'차량대수'})
mgrp5.loc[mgrp5.shape[0], ['구분', '차량대수']] = ['합계', mgrp5['차량대수'].sum()]
mgrp5

,구분,차종,차종유형,용도,차량대수
0,국내,승합,대형,개인용,34.0
1,국내,승합,대형,영업용,4.0
2,국내,화물,대형,개인용,291.0
3,국내,화물,대형,영업용,1773.0
4,국외,화물,대형,개인용,97.0
5,국외,화물,대형,영업용,173.0
6,합계,NaN,NaN,NaN,2372.0


## SCR장착 차량 현황

In [375]:
egrp1 = euro6.groupby(['구분'], dropna=False)['차대번호'].count().reset_index()
egrp1 = egrp1.rename(columns={'차대번호':'차량대수'})
egrp1.loc[egrp1.shape[0], ['구분', '차량대수']] = ['합계', egrp1['차량대수'].sum()]
egrp1

,구분,차량대수
0,국내,3592589.0
1,국외,840704.0
2,합계,4433293.0


In [376]:
egrp2 = euro6.groupby(['구분', '차종'], dropna=False)['차대번호'].count().reset_index()
egrp2 = egrp2.rename(columns={'차대번호':'차량대수'})
egrp2.loc[egrp2.shape[0], ['구분', '차량대수']] = ['합계', egrp2['차량대수'].sum()]
egrp2

,구분,차종,차량대수
0,국내,승용,1853816.0
1,국내,승합,198959.0
2,국내,특수,56761.0
3,국내,화물,1483053.0
4,국외,승용,763067.0
5,국외,승합,1562.0
6,국외,특수,23544.0
7,국외,화물,52531.0
8,합계,NaN,4433293.0


In [377]:
egrp3 = euro6.groupby(['구분', '차종유형'], dropna=False)['차대번호'].count().reset_index()
egrp3 = egrp3.rename(columns={'차대번호':'차량대수'})
egrp3.loc[egrp3.shape[0], ['구분', '차량대수']] = ['합계', egrp3['차량대수'].sum()]
egrp3

,구분,차종유형,차량대수
0,국내,대형,1169593.0
1,국내,소형,1317655.0
2,국내,중형,1105341.0
3,국외,대형,291046.0
4,국외,소형,4637.0
5,국외,중형,545021.0
6,합계,NaN,4433293.0


In [378]:
egrp4 = euro6.groupby(['구분', '용도'], dropna=False)['차대번호'].count().reset_index()
egrp4 = egrp4.rename(columns={'차대번호':'차량대수'})
egrp4.loc[egrp4.shape[0], ['구분', '차량대수']] = ['합계', egrp4['차량대수'].sum()]
egrp4

,구분,용도,차량대수
0,국내,개인용,3089301.0
1,국내,관용,41461.0
2,국내,영업용,461827.0
3,국외,개인용,765866.0
4,국외,관용,1736.0
5,국외,영업용,73102.0
6,합계,NaN,4433293.0


In [379]:
egrp5 = euro6.groupby(['구분', '차종', '차종유형', '용도'], dropna=False)['차대번호'].count().reset_index()
egrp5 = egrp5.rename(columns={'차대번호':'차량대수'})
egrp5.loc[egrp5.shape[0], ['구분', '차량대수']] = ['합계', egrp5['차량대수'].sum()]
egrp5

,구분,차종,차종유형,용도,차량대수
0,국내,승용,대형,개인용,884473.0
1,국내,승용,대형,관용,2186.0
2,국내,승용,대형,영업용,150335.0
3,국내,승용,소형,개인용,1.0
4,국내,승용,중형,개인용,761883.0
...,...,...,...,...,...
60,국외,화물,소형,영업용,6.0
61,국외,화물,중형,개인용,2007.0
62,국외,화물,중형,관용,20.0
63,국외,화물,중형,영업용,2898.0


### [출력] 요소수 차량 현황

In [ ]:
with pd.ExcelWriter(os.path.join(df_fold, f'요소수차량현황_{today_date}.xlsx')) as writer:
    mgrp1.to_excel(writer, sheet_name='저공해차량', index=False)
    mgrp2.to_excel(writer, sheet_name='저공해차량_차종', index=False)
    mgrp3.to_excel(writer, sheet_name='저공해차량_차종유형', index=False)
    mgrp4.to_excel(writer, sheet_name='저공해차량_용도', index=False)
    mgrp5.to_excel(writer, sheet_name='저공해차량_분류', index=False)
    egrp1.to_excel(writer, sheet_name='SCR장착차량', index=False)
    egrp2.to_excel(writer, sheet_name='SCR장착차량_차종', index=False)
    egrp3.to_excel(writer, sheet_name='SCR장착차량_차종유형', index=False)
    egrp4.to_excel(writer, sheet_name='SCR장착차량_용도', index=False)
    egrp5.to_excel(writer, sheet_name='SCR장착차량_분류', index=False)